#### Import packages and field data

In [1]:
from initialisation import *
Vs, Ts = {}, {}

# Import empirical vegetation density
Vs['grass'] = matrix_to_dict(np.loadtxt('field-data/grassland/vegetation.asc', skiprows=6)[1:-1,1:-1])
Vs['shrub'] = matrix_to_dict(np.loadtxt('field-data/shrubland/vegetation.asc', skiprows=6)[1:-1,1:-1])

# Import empirical topography
Ts['grass'] = matrix_to_dict(np.loadtxt('field-data/grassland/topography.asc', skiprows=6)[1:-1,1:-1])
Ts['shrub'] = matrix_to_dict(np.loadtxt('field-data/shrubland/topography.asc', skiprows=6)[1:-1,1:-1])

# Create baseline 2D lattice, empirical field data default width=20 and height=60
G = generate_baseline_lattice(20,60)


#### Linear regression between empirical vegetation density score and landscape microtopography

In [2]:
# Compute vegetation density scores on empirical shrubland
score_shrub = compute_landscape_vegetation_score(Vs['shrub'])

# Linear regression between X (scores) and Y (microtopography) to obtain a slope alpha, an intercept beta and a slope standard deviation alpha_std.
X, Y = [score_shrub[59-j,i] for i,j in G], [Ts['shrub'][i,j]-np.mean([Ts['shrub'][l,j] for l in range(20)]) for i,j in G]
alpha, beta = np.polyfit(X,Y,1)
Y_line = [i*alpha+beta for i in X]
alpha_std = np.std([Y[k]-Y_line[k] for k in range(len(G))])


KeyError: (20, 1)

#### Landscape generation

In [3]:
# Enter vegetation parameters
veg_cover, p_clustering = 0.3, 0.5

# Generate vegetation plot by sampling densities from the shrubland vegetation density distribution, for a cover veg_cover and clustering p_clustering
V = generate_vegetation(G, veg_cover, p_clustering, Vs['shrub'])

# Generate topography from the generated shrubland plot V
T = generate_topography(G, V, alpha, alpha_std, beta)


TypeError: unhashable type: 'list'

#### Visualisation of the generated landscape

In [ ]:
MT = [T[(i,j)]-np.mean([T[i,k] for k in range(59)]) for i,j in G]

fig, ax = plt.subplots(1,2, tight_layout=True)
p = [ax[0].imshow(dict_to_mat(V), cmap='Greens'), ax[1].imshow(dict_to_mat(MT), cmap='rainbow')]
_ = [ax[k].set_xticks([]) for k in range(2)]
_ = [ax[k].set_yticks([]) for k in range(2)]
cbar = [fig.colorbar(p[k], ax=ax[k], orientation='horizontal', shrink=.7) for k in range(2)]
cbar[0].set_label('vegetation density')
cbar[1].set_label('microtopography ($\\micro$ m)')
_ = ax[0].set_title('Generated\n vegetation')
_ = ax[1].set_title('Generated\n microtopography')
plt.show()


#### Reconstruction of shrubland microtopography as a test

In [ ]:
MT_shrub = [Ts['shrub'][(i,j)]-np.mean([Ts['shrub'][i,k] for k in range(59)]) for i,j in G]
T_shrub_reconstructed = generate_topography(G, Vs['shrub'], alpha, alpha_std, beta)
MT_shrub_reconstructed = [T_shrub_reconstructed[(i,j)]-np.mean([T_shrub_reconstructed[i,k] for k in range(59)]) for i,j in G]

fig, ax = plt.subplots(1,3, tight_layout=True)
p = [ax[0].imshow(dict_to_mat(V), cmap='Greens'), ax[1].imshow(dict_to_mat(MT), cmap='rainbow'), ax[2].imshow(dict_to_mat(MT), cmap='rainbow')]
_ = [ax[k].set_xticks([]) for k in range(3)]
_ = [ax[k].set_yticks([]) for k in range(3)]
cbar = [fig.colorbar(p[k], ax=ax[k], orientation='horizontal', shrink=.7) for k in range(3)]
cbar[0].set_label('vegetation density')
cbar[1].set_label('microtopography ($\\micro$ m)')
cbar[2].set_label('microtopography ($\\micro$ m)')
_ = ax[0].set_title('Shrubland vegetation')
_ = ax[1].set_title('Original shrubland\nmicrotopography')
_ = ax[2].set_title('Reconstructed shrubland\nmicrotopography')
plt.show()
